In [1]:
# Imports
import os 
import pandas as pd
from django.core.exceptions import ValidationError
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from datetime import date
from edc_appointment.models import Appointment

In [2]:
OnScheduleCohortCSec.objects.get(subject_identifier='B142-040990001-5', schedule_name='c_sec2_schedule1')

DoesNotExist: OnScheduleCohortCSec matching query does not exist.

In [2]:
OnScheduleCohortCSec.objects.get(subject_identifier='B142-040990001-5', schedule_name='c_sec2_schedule1').delete()

OnScheduleCohortCSec.objects.get(subject_identifier='B142-040990005-6', schedule_name='c_sec2_schedule1').delete()

OnScheduleCohortCSec.objects.get(subject_identifier='B142-040990003-1', schedule_name='c_sec2_schedule1').delete()


(1, {'flourish_caregiver.OnScheduleCohortCSec': 1})

In [3]:
count = 0
dups = []
success = []
def update_window(sch, pid):
    subject_identifier = pid
    print('\n\n\n')
    print(";;;;;;;;", pid)
    
    onses = sch.objects.filter(subject_identifier=subject_identifier)
    
    for ons in onses:
        sn = ons.schedule_name
        
        try:
            ap0 = Appointment.objects.get(subject_identifier=subject_identifier,
                                          visit_code_sequence=0,
                                          visit_code='2000M')
        except Appointment.MultipleObjectsReturned:
            dups.append(subject_identifier)
        else:
            ons.schedule_name = ons.schedule_name + 'x'
            ons.save()


            try:
                ap0 = Appointment.objects.get(subject_identifier=subject_identifier,
                                              schedule_name=sn,
                                              visit_code_sequence=0,
                                              visit_code='2000M')
            except Appointment.DoesNotExist:
                print(":::::::::::::::::", sn, subject_identifier)

            ap0.schedule_name = ons.schedule_name
            ap0.save()

            # resave enroll visit
            cpe = CaregiverPreviouslyEnrolled.objects.get(subject_identifier=subject_identifier)

            cpe.save()

    #         import pdb; pdb.set_trace()
            ap1 = Appointment.objects.get(subject_identifier=subject_identifier,
                                          schedule_name=sn,
                                          visit_code_sequence=0,
                                         visit_code='2000M')

            ap1.user_modified = ap0.user_modified
            ap1.appt_status = ap0.appt_status
            ap1.appt_reason = ap0.appt_reason
            ap1.appt_datetime = ap0.appt_datetime

            ap1.save()

            try:
                v1 = MaternalVisit.objects.get(appointment=ap0)
                v1.appointment = ap1
                v1.save()
            except:
                pass

            ap0.delete()
            ons.delete()
            success.append(subject_identifier)

In [9]:
##Maternal Window
from edc_appointment.models import Appointment

pids = []

for g in [OnScheduleCohortAEnrollment, OnScheduleCohortBEnrollment, OnScheduleCohortCEnrollment,
         OnScheduleCohortASec, OnScheduleCohortBSec, OnScheduleCohortCSec]:
    
    inner_pids = []
    for a in g.objects.all():
        try:
            prev_enrol = CaregiverPreviouslyEnrolled.objects.get(subject_identifier=a.subject_identifier,)
        except CaregiverPreviouslyEnrolled.DoesNotExist:
            pass
        else:
            if prev_enrol.report_datetime.date() != a.onschedule_datetime.date():
                pids.append([a.subject_identifier, g])
                inner_pids.append(a.subject_identifier)
            
    


In [11]:
pids

[['B142-040990044-5',
  flourish_caregiver.models.onschedule.OnScheduleCohortBEnrollment],
 ['B142-040990623-6',
  flourish_caregiver.models.onschedule.OnScheduleCohortCEnrollment],
 ['B142-040990023-9',
  flourish_caregiver.models.onschedule.OnScheduleCohortBSec],
 ['B142-040990602-0',
  flourish_caregiver.models.onschedule.OnScheduleCohortBSec],
 ['B142-040990612-9',
  flourish_caregiver.models.onschedule.OnScheduleCohortCSec],
 ['B142-040990002-3',
  flourish_caregiver.models.onschedule.OnScheduleCohortCSec],
 ['B142-040990606-1',
  flourish_caregiver.models.onschedule.OnScheduleCohortCSec],
 ['B142-040990007-2',
  flourish_caregiver.models.onschedule.OnScheduleCohortCSec]]

In [10]:
len(pids)

8

In [6]:
failed_pids = []
for pid, sch in pids:
        consents = SubjectConsent.objects.filter(subject_identifier=pid)
#         try:
#             FlourishConsentVersion.objects.get(screening_identifier=consents[0].screening_identifier)
#         except FlourishConsentVersion.DoesNotExist:
#             failed_pids.append(a.subject_identifier)
#         else:
        try:
            update_window(sch, pid)
        except Exception as e:
            print(e)
            failed_pids.append(pid)





;;;;;;;; B142-040990504-8




;;;;;;;; B142-040990581-6




;;;;;;;; B142-040990006-4




;;;;;;;; B142-040990009-8




;;;;;;;; B142-040990044-5




;;;;;;;; B142-040990585-7




;;;;;;;; B142-040990008-0




;;;;;;;; B142-040990623-6
Associated caregiver consent on behalf of child for this participant not found




;;;;;;;; B142-040990023-9




;;;;;;;; B142-040990602-0
Associated caregiver consent on behalf of child for this participant not found




;;;;;;;; B142-040990263-1




;;;;;;;; B142-040990612-9
Associated caregiver consent on behalf of child for this participant not found




;;;;;;;; B142-040990002-3
::::::::::::::::: c_sec2_schedule1 B142-040990002-3
Appointment matching query does not exist.




;;;;;;;; B142-040990001-5




;;;;;;;; B142-040990003-1




;;;;;;;; B142-040990005-6




;;;;;;;; B142-040990182-3




;;;;;;;; B142-040990002-3




;;;;;;;; B142-040990606-1
Associated caregiver consent on behalf of child for this participant not found




;;;;;;;; B142-0

In [7]:
len(success)

12

In [8]:
len(failed_pids)

6

In [10]:
failed_pids

['B142-040990623-6',
 'B142-040990602-0',
 'B142-040990612-9',
 'B142-040990002-3',
 'B142-040990606-1',
 'B142-040990007-2']

In [12]:
# success

In [4]:
# for pid, sch in [['B142-040990044-5', OnScheduleCohortBEnrollment],]:

subject_identifier = 'B142-040990623-6'
onses = OnScheduleCohortCEnrollment.objects.filter(subject_identifier='B142-040990623-6')
if onses.count() > 1:
    print("lllllll", subject_identifier)

for ons in onses:
    sn = ons.schedule_name

    ons.schedule_name = ons.schedule_name + 'x'
    ons.save()
    
    import pdb; pdb.set_trace()

    try:
        ap0 = Appointment.objects.get(subject_identifier=subject_identifier,
                                      schedule_name=sn,
                                      visit_code_sequence=0,
                                      visit_code='2000M')
    except Appointment.DoesNotExist:
        print(":::::::::::::::::", sn, subject_identifier)

    ap0.schedule_name = ons.schedule_name
    ap0.save()

     # resave enroll visit
    ccc = CaregiverChildConsent.objects.get(subject_identifier=ons.child_subject_identifier)

    ccc.save()

#         import pdb; pdb.set_trace()
    print('}}}}}}}}}}}}', sn)
    import pdb; pdb.set_trace()
    ap1 = Appointment.objects.get(subject_identifier=subject_identifier,
                                  schedule_name=sn,
                                  visit_code_sequence=0,
                                 visit_code='2000M')

    ap1.user_modified = ap0.user_modified
    ap1.appt_status = ap0.appt_status
    ap1.appt_reason = ap0.appt_reason
    ap1.appt_datetime = ap0.appt_datetime

    ap1.save()

    try:
        v1 = MaternalVisit.objects.get(appointment=ap0)
        v1.appointment = ap1
        v1.save()
    except:
        pass

    ap0.delete()
    ons.delete()


> <ipython-input-4-3751e20ed051>(16)<module>()
     14     import pdb; pdb.set_trace()
     15 
---> 16     try:
     17         ap0 = Appointment.objects.get(subject_identifier=subject_identifier,
     18                                       schedule_name=sn,

ipdb> n
> <ipython-input-4-3751e20ed051>(17)<module>()
     15 
     16     try:
---> 17         ap0 = Appointment.objects.get(subject_identifier=subject_identifier,
     18                                       schedule_name=sn,
     19                                       visit_code_sequence=0,

ipdb> n
> <ipython-input-4-3751e20ed051>(18)<module>()
     16     try:
     17         ap0 = Appointment.objects.get(subject_identifier=subject_identifier,
---> 18                                       schedule_name=sn,
     19                                       visit_code_sequence=0,
     20                                       visit_code='2000M')

ipdb> n
> <ipython-input-4-3751e20ed051>(19)<module>()
     17         ap0 = App

ipdb> n
> <ipython-input-4-3751e20ed051>(36)<module>()
     34     import pdb; pdb.set_trace()
     35     ap1 = Appointment.objects.get(subject_identifier=subject_identifier,
---> 36                                   schedule_name=sn,
     37                                   visit_code_sequence=0,
     38                                  visit_code='2000M')

ipdb> n
> <ipython-input-4-3751e20ed051>(37)<module>()
     35     ap1 = Appointment.objects.get(subject_identifier=subject_identifier,
     36                                   schedule_name=sn,
---> 37                                   visit_code_sequence=0,
     38                                  visit_code='2000M')
     39 

ipdb> n
> <ipython-input-4-3751e20ed051>(38)<module>()
     36                                   schedule_name=sn,
     37                                   visit_code_sequence=0,
---> 38                                  visit_code='2000M')
     39 
     40     ap1.user_modified = ap0.user_modified

ipdb

ipdb> n
> <ipython-input-4-3751e20ed051>(50)<module>()
     48         v1 = MaternalVisit.objects.get(appointment=ap0)
     49         v1.appointment = ap1
---> 50         v1.save()
     51     except:
     52         pass

ipdb> n
> <ipython-input-4-3751e20ed051>(54)<module>()
     51     except:
     52         pass
     53 
---> 54     ap0.delete()
     55     ons.delete()

ipdb> 
> <ipython-input-4-3751e20ed051>(55)<module>()
     51     except:
     52         pass
     53 
     54     ap0.delete()
---> 55     ons.delete()

ipdb> n
> <ipython-input-4-3751e20ed051>(8)<module>()
      6     print("lllllll", subject_identifier)
      7 
----> 8 for ons in onses:
      9     sn = ons.schedule_name
     10 

ipdb> onses
<QuerySet [<OnScheduleCohortCEnrollment: B142-040990623-6 02/08/2022>]>
ipdb> n
--Return--
None
> <ipython-input-4-3751e20ed051>(8)<module>()
      6     print("lllllll", subject_identifier)
      7 
----> 8 for ons in onses:
      9     sn = ons.schedule_name
     10 


ipdb> n
> /Users/imosweu/.venvs/flourish-live/lib/python3.9/site-packages/IPython/core/interactiveshell.py(3175)run_cell_async()
   3173                 self.displayhook.exec_result = None
   3174 
-> 3175         if store_history:
   3176             # Write output to the database. Does nothing unless
   3177             # history output logging is enabled.

ipdb> n
> /Users/imosweu/.venvs/flourish-live/lib/python3.9/site-packages/IPython/core/interactiveshell.py(3178)run_cell_async()
   3176             # Write output to the database. Does nothing unless
   3177             # history output logging is enabled.
-> 3178             self.history_manager.store_output(self.execution_count)
   3179             # Each cell is a *single* input, regardless of how many lines it has
   3180             self.execution_count += 1

ipdb> n
> /Users/imosweu/.venvs/flourish-live/lib/python3.9/site-packages/IPython/core/interactiveshell.py(3180)run_cell_async()
   3178             self.history_manage

In [22]:
ap1, ap2, ap3 = Appointment.objects.filter(subject_identifier='B142-040990044-5', visit_code='2000M')


In [24]:
ap3.__dict__

{'_state': <django.db.models.base.ModelState at 0x13470eaf0>,
 'created': datetime.datetime(2021, 5, 28, 13, 3, 42, 473363, tzinfo=<UTC>),
 'modified': datetime.datetime(2022, 3, 25, 13, 58, 56, 321574, tzinfo=<UTC>),
 'user_created': 'flourish',
 'user_modified': 'skgole',
 'hostname_created': 'flourish',
 'hostname_modified': 'Itumelengs-MacBook-Pro.local',
 'revision': '',
 'device_created': '99',
 'device_modified': '99',
 'id': UUID('5234ac74-98ae-44bf-9567-0ffc1ea8444a'),
 'site_id': 40,
 'subject_identifier': 'B142-040990044-5',
 'visit_schedule_name': 'c1_visit_schedule1',
 'schedule_name': 'c_sec1_schedule1',
 'visit_code': '2000M',
 'visit_code_sequence': 0,
 'timepoint_status': 'open',
 'timepoint_opened_datetime': datetime.datetime(2021, 5, 28, 13, 4, 5, tzinfo=<UTC>),
 'timepoint_closed_datetime': None,
 'timepoint': Decimal('0.0'),
 'timepoint_datetime': datetime.datetime(2021, 5, 28, 13, 4, 5, tzinfo=<UTC>),
 'appt_close_datetime': None,
 'facility_name': '5-day clinic',